In [ ]:
import json
import pandas as pd
from gensim.models import Word2Vec
from konlpy.tag import Komoran
import time

data = []
labels = []
p = []
with open('/mnt/c/src/chatgpt/용례_게임.json', 'r') as f:
    json_data = json.load(f)
    for i in json_data:
        for j in i["tokens"]:
            a = j['sub']
            data.append(a)
            label = j["facet"]
            if label not in p:
                p.append(label)
            labels.append(p.index(label))
            #labels.append(label)

df = pd.DataFrame({'Column1': data, 'Column2': labels})
print(df.head(3))
print(len(p))
df.to_csv('./game_data.csv', index=False)


In [ ]:
# 필요한 모듈 임포트
import pandas as pd
import tensorflow as tf
from tensorflow.keras import preprocessing
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Dense, Dropout, Conv1D, GlobalMaxPool1D, concatenate
from sklearn.model_selection import train_test_split
# 데이터 읽어오기

train_file = "./game_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Column1'].tolist()
labels = data['Column2'].tolist()

# 단어 인덱스 시퀀스 벡터
corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
word_index = tokenizer.word_index
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

# 데이터셋 생성
train_features, test_features, train_labels, test_labels = train_test_split(padded_seqs, labels, test_size=0.1, random_state=42)
train_features, val_features, train_labels, val_labels = train_test_split(train_features, train_labels, test_size=0.1, random_state=42)

# Define datasets
train_ds = tf.data.Dataset.from_tensor_slices((train_features, train_labels)).shuffle(buffer_size=10000).batch(64)
val_ds = tf.data.Dataset.from_tensor_slices((val_features, val_labels)).batch(64)
test_ds = tf.data.Dataset.from_tensor_slices((test_features, test_labels)).batch(64)


# 하이퍼파라미터 설정
dropout_prob = 0.5
EMB_SIZE = 128
EPOCH = 5
VOCAB_SIZE = len(word_index) + 1 # 전체 단어 수
# CNN 모델 정의
input_layer = Input(shape=(MAX_SEQ_LEN,))
embedding_layer = Embedding(VOCAB_SIZE, EMB_SIZE,
input_length=MAX_SEQ_LEN)(input_layer)
dropout_emb = Dropout(rate=dropout_prob)(embedding_layer)

conv1 = Conv1D(filters=128, kernel_size=3, padding='valid',
activation=tf.nn.relu)(dropout_emb)
pool1 = GlobalMaxPool1D()(conv1)
conv2 = Conv1D(filters=128, kernel_size=4, padding='valid',
activation=tf.nn.relu)(dropout_emb)
pool2 = GlobalMaxPool1D()(conv2)
conv3 = Conv1D(filters=128, kernel_size=5, padding='valid',
activation=tf.nn.relu)(dropout_emb)
pool3 = GlobalMaxPool1D()(conv3)
# 합치기
concat = concatenate([pool1, pool2, pool3])

hidden = Dense(128, activation=tf.nn.relu)(concat)
dropout_hidden = Dropout(rate=dropout_prob)(hidden)
logits = Dense(19, name='logits')(dropout_hidden)


predictions = Dense(19, activation=tf.nn.softmax)(logits)

# 모델 생성
model = Model(inputs=input_layer, outputs=predictions)
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(train_ds, validation_data=val_ds, epochs=EPOCH, verbose=1)

# 모델 평가(테스트 데이터셋 이용)
loss, accuracy = model.evaluate(test_ds, verbose=1)
print('Accuracy: %f' % (accuracy * 100))
print('loss: %f' % (loss))

model.save('cnn_model_prac.h5')

In [ ]:
import tensorflow as tf
import pandas as pd
from tensorflow.keras.models import Model, load_model
from tensorflow.keras import preprocessing
import json


p = []
with open('/mnt/c/src/chatgpt/용례_게임.json', 'r') as f:
    json_data = json.load(f)
    for i in json_data:
        for j in i["tokens"]:
            label = j["facet"]
            if label not in p:
                p.append(label)



# 데이터 읽어오기
train_file = "./game_data.csv"
data = pd.read_csv(train_file, delimiter=',')
features = data['Column1'].tolist()
labels = data['Column2'].tolist()
a = data['Column2'].unique()


corpus = [preprocessing.text.text_to_word_sequence(text) for text in features]
tokenizer = preprocessing.text.Tokenizer()
tokenizer.fit_on_texts(corpus)
sequences = tokenizer.texts_to_sequences(corpus)
MAX_SEQ_LEN = 15 # 단어 시퀀스 벡터 크기
padded_seqs = preprocessing.sequence.pad_sequences(sequences, maxlen=MAX_SEQ_LEN, padding='post')

ds = tf.data.Dataset.from_tensor_slices((padded_seqs, labels))
ds = ds.shuffle(len(features))
test_ds = ds.take(2000).batch(20) # 테스트 데이터셋

model = load_model('cnn_model_prac.h5')
model.summary()
model.evaluate(test_ds, verbose=2)

print("단어 시퀀스 : ", corpus[10212])
#print("단어 인덱스 시퀀스 : ", padded_seqs[10212])
print("문장 분류(정답) : ", labels[10212], p[2])

picks = [10212]
predict = model.predict(padded_seqs[picks])
predict_class = tf.math.argmax(predict, axis=1)
print(type(predict_class))
print("예측 점수 : ", predict)
print("예측 클래스 : ", predict_class.numpy(), p[int(predict_class)])